In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import holidays
import re
from sklearn.ensemble import RandomForestRegressor

file_path = 'Buildings_el.xlsx'
df_weather = pd.read_excel(file_path, sheet_name='Weather archive', skiprows=2, parse_dates=['Local time in Tallinn '])
df_weather.rename(columns={'Local time in Tallinn ': 'Timestamp'}, inplace=True)
df_weather["Timestamp"] = pd.to_datetime(df_weather["Timestamp"], format="%d.%m.%Y %H:%M", dayfirst=True)
df_weather = df_weather.sort_values(by='Timestamp')
#Limit time to match electricity data available
df_weather = df_weather[df_weather['Timestamp']<="2023-12-31 23:55"]

df_el = pd.read_excel(file_path, sheet_name = 'Electricity kWh', skiprows=1, parse_dates=['Timestamp'])
df_el["Timestamp"] = pd.to_datetime(df_el["Timestamp"], format="%d.%m.%Y %H:%M", dayfirst=True)

df_area = pd.read_excel(file_path, sheet_name='Areas')
df_area = df_area.rename(columns={'Buid_ID':'building'})

df_weather.head()

,Timestamp,T,P0,P,U,DD,Ff,ff10,WW,W'W',c,VV,Td
17535,2023-01-01 00:20:00,5.0,742.6,746.3,93.0,Wind blowing from the south-southwest,6,NaN,NaN,NaN,"Scattered clouds (40-50%) 240 m, broken clouds...",9,4.0
17534,2023-01-01 00:50:00,5.0,742.6,746.3,93.0,Wind blowing from the south-southwest,4,NaN,NaN,NaN,"Few clouds (10-30%) 240 m, scattered clouds (4...",10.0 and more,4.0
17533,2023-01-01 01:20:00,5.0,742.6,746.3,87.0,Wind blowing from the south-west,5,NaN,NaN,NaN,"Scattered clouds (40-50%) 390 m, broken clouds...",10.0 and more,3.0
17532,2023-01-01 01:50:00,5.0,743.3,747.0,93.0,Wind blowing from the west-southwest,5,NaN,NaN,NaN,Broken clouds (60-90%) 360 m,10.0 and more,4.0
17531,2023-01-01 02:20:00,5.0,743.3,747.0,93.0,Wind blowing from the west-southwest,6,NaN,NaN,NaN,Broken clouds (60-90%) 330 m,10.0 and more,4.0


In [3]:
# Fill NaNs in 'c' column before processing, assuming NaN means clear or no significant clouds
df_weather['c'] = df_weather['c'].fillna("no significant clouds")

# Create cloud_condition_score feature based on 'c' column text

# Define mapping function for cloud condition score: ordinal from best (clear=1) to worst (overcast=5)
def map_cloud_condition(text):
    t = str(text).lower()
    if 'overcast' in t:
        return 5
    elif 'broken clouds' in t:
        return 4
    elif 'scattered clouds' in t:
        return 3
    elif 'few clouds' in t:
        return 2
    elif 'no significant clouds' in t: # Handles explicit clear sky and filled NaNs
        return 1
    else:
        # Default for any other unrecognized text, assuming clear if not specified otherwise
        return 1

# Apply the function to create the new feature
df_weather['cloud_condition_score'] = df_weather['c'].apply(map_cloud_condition)

# Display value counts for the new feature
print("Value counts for cloud_condition_score:")
print(df_weather['cloud_condition_score'].value_counts())



Value counts for cloud_condition_score:
cloud_condition_score
1    6307
5    4713
4    2915
2    2003
3    1550
Name: count, dtype: int64


In [4]:
# Extract numerical values (meters) from the 'c' column using regular expressions
# This extracts the first occurrence of digits followed by ' m'
df_weather['cloud_base_meters'] = df_weather['c'].str.extract(r'(\d+)\s*m')

# Convert the extracted column to numeric, setting non-extractable entries to NaN
df_weather['cloud_base_meters'] = pd.to_numeric(df_weather['cloud_base_meters'], errors='coerce')

# Display the original 'c' column and the new 'cloud_base_meters' column for comparison
print(df_weather[['c', 'cloud_base_meters']].head())

# Display summary statistics for the new column
print("\nSummary statistics for 'cloud_base_meters':")
print(df_weather['cloud_base_meters'].describe())

                                                       c  cloud_base_meters
17535  Scattered clouds (40-50%) 240 m, broken clouds...              240.0
17534  Few clouds (10-30%) 240 m, scattered clouds (4...              240.0
17533  Scattered clouds (40-50%) 390 m, broken clouds...              390.0
17532                       Broken clouds (60-90%) 360 m              360.0
17531                       Broken clouds (60-90%) 330 m              330.0

Summary statistics for 'cloud_base_meters':
count    11396.000000
mean       539.065462
std        372.075444
min         30.000000
25%        210.000000
50%        450.000000
75%        810.000000
max       1500.000000
Name: cloud_base_meters, dtype: float64


In [5]:
df_weather["T"] = df_weather["T"].ffill().bfill()
df_weather["P0"] = df_weather["P0"].ffill().bfill()
df_weather["P"] = df_weather["P"].ffill().bfill()
df_weather["U"] = df_weather["U"].ffill().bfill()
df_weather["Td"] = df_weather["Td"].ffill().bfill()
df_weather["c"] = df_weather["c"].ffill().bfill()
df_weather["cloud_base_meters"] = df_weather["cloud_base_meters"].ffill().bfill()
df_weather["ff10"] = df_weather["ff10"].fillna(df_weather['Ff'])
df_weather.drop("W'W'", axis=1, inplace=True)
df_weather.drop("c", axis=1, inplace=True)
df_weather.head()

,Timestamp,T,P0,P,U,DD,Ff,ff10,WW,VV,Td,cloud_condition_score,cloud_base_meters
17535,2023-01-01 00:20:00,5.0,742.6,746.3,93.0,Wind blowing from the south-southwest,6,6.0,NaN,9,4.0,4,240.0
17534,2023-01-01 00:50:00,5.0,742.6,746.3,93.0,Wind blowing from the south-southwest,4,4.0,NaN,10.0 and more,4.0,4,240.0
17533,2023-01-01 01:20:00,5.0,742.6,746.3,87.0,Wind blowing from the south-west,5,5.0,NaN,10.0 and more,3.0,4,390.0
17532,2023-01-01 01:50:00,5.0,743.3,747.0,93.0,Wind blowing from the west-southwest,5,5.0,NaN,10.0 and more,4.0,4,360.0
17531,2023-01-01 02:20:00,5.0,743.3,747.0,93.0,Wind blowing from the west-southwest,6,6.0,NaN,10.0 and more,4.0,4,330.0


In [6]:
df_weather.isnull().sum()

Timestamp                    0
T                            0
P0                           0
P                            0
U                            0
DD                           0
Ff                           0
ff10                         0
WW                       13438
VV                           0
Td                           0
cloud_condition_score        0
cloud_base_meters            0
dtype: int64

In [7]:
df_weather['VV'] = df_weather["VV"].replace('10.0 and more', '10.1').astype(float)
df_weather.head()

,Timestamp,T,P0,P,U,DD,Ff,ff10,WW,VV,Td,cloud_condition_score,cloud_base_meters
17535,2023-01-01 00:20:00,5.0,742.6,746.3,93.0,Wind blowing from the south-southwest,6,6.0,NaN,9.0,4.0,4,240.0
17534,2023-01-01 00:50:00,5.0,742.6,746.3,93.0,Wind blowing from the south-southwest,4,4.0,NaN,10.1,4.0,4,240.0
17533,2023-01-01 01:20:00,5.0,742.6,746.3,87.0,Wind blowing from the south-west,5,5.0,NaN,10.1,3.0,4,390.0
17532,2023-01-01 01:50:00,5.0,743.3,747.0,93.0,Wind blowing from the west-southwest,5,5.0,NaN,10.1,4.0,4,360.0
17531,2023-01-01 02:20:00,5.0,743.3,747.0,93.0,Wind blowing from the west-southwest,6,6.0,NaN,10.1,4.0,4,330.0


In [8]:
df_weather['DD'].value_counts()

DD
Wind blowing from the south-west         1570
Wind blowing from the west               1501
Wind blowing from the east               1500
Wind blowing from the south-southwest    1493
Wind blowing from the south              1485
Wind blowing from the west-southwest     1101
Wind blowing from the north-west          945
Wind blowing from the east-southeast      910
Wind blowing from the west-northwest      903
variable wind direction                   859
Wind blowing from the south-east          780
Wind blowing from the north               759
Wind blowing from the east-northeast      754
Wind blowing from the south-southeast     749
Wind blowing from the north-northwest     709
Wind blowing from the north-east          644
Wind blowing from the north-northeast     560
Calm, no wind                             266
Name: count, dtype: int64

In [9]:
# Mapping wind directions to angles
direction_map = {
    "Wind blowing from the north": 0,
    "Wind blowing from the north-northeast": 22.5,
    "Wind blowing from the north-east": 45,
    "Wind blowing from the east-northeast": 67.5,
    "Wind blowing from the east": 90,
    "Wind blowing from the east-southeast": 112.5,
    "Wind blowing from the south-east": 135,
    "Wind blowing from the south-southeast": 157.5,
    "Wind blowing from the south": 180,
    "Wind blowing from the south-southwest": 202.5,
    "Wind blowing from the south-west": 225,
    "Wind blowing from the west-southwest": 247.5,
    "Wind blowing from the west": 270,
    "Wind blowing from the west-northwest": 292.5,
    "Wind blowing from the north-west": 315,
    "Wind blowing from the north-northwest": 337.5,
    "variable wind direction": -1,
    "Calm, no wind": -2
}

# Convert to degree value
df_weather["wind_direction_deg"] = df_weather["DD"].map(direction_map)
df_weather['wind_direction_rad'] = np.deg2rad(df_weather['wind_direction_deg'])

df_weather['wind_x'] = np.cos(df_weather['wind_direction_rad'])
df_weather['wind_y'] = np.sin(df_weather['wind_direction_rad'])


# Optional: Add flags
df_weather["wind_is_variable"] = (df_weather["DD"] == "variable wind direction").astype(int)
df_weather["wind_is_calm"] = (df_weather["DD"] == "Calm, no wind").astype(int)
df_weather.drop('DD', axis=1, inplace=True)
df_weather.drop('wind_direction_deg', axis=1, inplace=True)
df_weather.drop('wind_direction_rad', axis=1, inplace=True)


df_weather

,Timestamp,T,P0,P,U,Ff,ff10,WW,VV,Td,cloud_condition_score,cloud_base_meters,wind_x,wind_y,wind_is_variable,wind_is_calm
17535,2023-01-01 00:20:00,5.0,742.6,746.3,93.0,6,6.0,NaN,9.0,4.0,4,240.0,-9.238795e-01,-0.382683,0,0
17534,2023-01-01 00:50:00,5.0,742.6,746.3,93.0,4,4.0,NaN,10.1,4.0,4,240.0,-9.238795e-01,-0.382683,0,0
17533,2023-01-01 01:20:00,5.0,742.6,746.3,87.0,5,5.0,NaN,10.1,3.0,4,390.0,-7.071068e-01,-0.707107,0,0
17532,2023-01-01 01:50:00,5.0,743.3,747.0,93.0,5,5.0,NaN,10.1,4.0,4,360.0,-3.826834e-01,-0.923880,0,0
17531,2023-01-01 02:20:00,5.0,743.3,747.0,93.0,6,6.0,NaN,10.1,4.0,4,330.0,-3.826834e-01,-0.923880,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,2023-12-31 21:50:00,-8.0,758.9,762.8,93.0,0,0.0,NaN,10.1,-9.0,4,600.0,9.993908e-01,-0.034899,0,1
51,2023-12-31 22:20:00,-8.0,758.9,762.8,93.0,0,0.0,"Light shower(s), snow",6.0,-9.0,4,570.0,9.993908e-01,-0.034899,0,1
50,2023-12-31 22:50:00,-8.0,759.6,763.5,93.0,1,1.0,"Light shower(s), snow",10.1,-9.0,4,540.0,9.998477e-01,-0.017452,1,0
49,2023-12-31 23:20:00,-8.0,759.6,763.5,93.0,1,1.0,"Light shower(s), snow",6.0,-9.0,4,510.0,-1.836970e-16,-1.000000,0,0


In [10]:
df_weather['WW'].unique()

array([nan, 'Light snow, rain', 'Light snow', 'Light rain, snow',
       'Light rain', 'Snow', 'Shower(s), small hail and/or snow pellets',
       'Shower(s), snow', 'Heavy shower(s), snow', 'Light snow, mist',
       'Snow, mist', 'Light shower(s), snow',
       'Light shower(s), snow, low drifting, snow',
       'Shower(s), snow, low drifting, snow',
       'Heavy shower(s), snow, low drifting, snow',
       'Shower(s), snow,blowing snow', 'Low drifting, snow',
       'Light snow, low drifting, snow', 'Snow, low drifting, snow',
       'Heavy snow, low drifting, snow', 'Heavy snow', 'Mist',
       'Light freezing (supercooled) drizzle, mist',
       'Light drizzle, mist', 'Light rain, mist', 'Fog',
       'Light drizzle, mist,patches fog', 'Mist,patches fog',
       'Light drizzle', 'Snow, rain', 'Light rain, drizzle',
       'Light rain, drizzle, mist', 'Light shower(s), rain', 'Drizzle',
       'Drizzle, mist', 'Shower(s), rain', 'Light drizzle, rain, mist',
       'Light shower(s)

In [11]:

# Define keywords and intensity weights
weather_weights = {
    'fog': 1,
    'mist': 1,
    'drizzle': 2,
    'rain': 3,
    'snow': 4,
    'low drifting': 5,
    'blowing snow': 5,
    'hail': 6,
    'freezing': 7,
    'thunderstorm': 8
}
# Define sets for binary flags
precip_keywords = ['rain', 'snow', 'drizzle']
severe_keywords = ['thunderstorm', 'freezing', 'hail']
low_vis_keywords = ['fog', 'mist']

# Function to compute all weather features
def extract_weather_features(text):
    if pd.isna(text):
        return pd.Series({
            'weather_severity': 1,
            # 'is_precipitation': 0,
            # 'is_severe_weather': 0,
            # 'is_low_visibility': 0
        })

    text = text.lower()
    score = 1
    is_precip = 0
    is_severe = 0
    is_low_vis = 0

    for kw, w in weather_weights.items():
        if kw in text:
            score += w
    for kw in precip_keywords:
        if kw in text:
            is_precip = 1
    for kw in severe_keywords:
        if kw in text:
            is_severe = 1
    for kw in low_vis_keywords:
        if kw in text:
            is_low_vis = 1

    return pd.Series({
        'weather_severity': score,
        # 'is_precipitation': is_precip,
        # 'is_severe_weather': is_severe,
        # 'is_low_visibility': is_low_vis
    })

weather_features = df_weather['WW'].apply(extract_weather_features)
df_weather = pd.concat([df_weather, weather_features], axis=1)
df_weather.drop(columns=['WW'], inplace=True)
df_weather



,Timestamp,T,P0,P,U,Ff,ff10,VV,Td,cloud_condition_score,cloud_base_meters,wind_x,wind_y,wind_is_variable,wind_is_calm,weather_severity
17535,2023-01-01 00:20:00,5.0,742.6,746.3,93.0,6,6.0,9.0,4.0,4,240.0,-9.238795e-01,-0.382683,0,0,1
17534,2023-01-01 00:50:00,5.0,742.6,746.3,93.0,4,4.0,10.1,4.0,4,240.0,-9.238795e-01,-0.382683,0,0,1
17533,2023-01-01 01:20:00,5.0,742.6,746.3,87.0,5,5.0,10.1,3.0,4,390.0,-7.071068e-01,-0.707107,0,0,1
17532,2023-01-01 01:50:00,5.0,743.3,747.0,93.0,5,5.0,10.1,4.0,4,360.0,-3.826834e-01,-0.923880,0,0,1
17531,2023-01-01 02:20:00,5.0,743.3,747.0,93.0,6,6.0,10.1,4.0,4,330.0,-3.826834e-01,-0.923880,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,2023-12-31 21:50:00,-8.0,758.9,762.8,93.0,0,0.0,10.1,-9.0,4,600.0,9.993908e-01,-0.034899,0,1,1
51,2023-12-31 22:20:00,-8.0,758.9,762.8,93.0,0,0.0,6.0,-9.0,4,570.0,9.993908e-01,-0.034899,0,1,5
50,2023-12-31 22:50:00,-8.0,759.6,763.5,93.0,1,1.0,10.1,-9.0,4,540.0,9.998477e-01,-0.017452,1,0,5
49,2023-12-31 23:20:00,-8.0,759.6,763.5,93.0,1,1.0,6.0,-9.0,4,510.0,-1.836970e-16,-1.000000,0,0,5


In [12]:
df_weather.set_index("Timestamp", inplace=True)

full_hour_range = pd.date_range(
    start=df_el["Timestamp"].min(),
    end=df_el["Timestamp"].max(),
    freq="1H"
)

df_weather_hourly = df_weather.resample("1H").mean()
df_weather_hourly = df_weather_hourly.reindex(full_hour_range)
df_weather_hourly.index.name = "Timestamp"


numeric_cols = [
    "T", "P0", "P", "U", "Ff", "ff10", "VV", "Td",
    "cloud_base_meters", "cloud_condition_score",
    "wind_x", "wind_y", "weather_severity",
]

binary_cols = [col for col in df_weather_hourly.columns
               if col in ["wind_is_variable", "wind_is_calm",]]

df_weather_hourly[numeric_cols] = df_weather_hourly[numeric_cols].interpolate(method="time")

df_weather_hourly[binary_cols] = df_weather_hourly[binary_cols].fillna(0)

df_weather_hourly.fillna(method="ffill", inplace=True)
df_weather_hourly.fillna(method="bfill", inplace=True)

df_weather_hourly = df_weather_hourly.reset_index()

df_weather.reset_index(inplace=True)
df_weather_hourly.isnull().sum()

C:\Users\ashooshtari\AppData\Local\Temp\ipykernel_17108\3292534948.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_hour_range = pd.date_range(
C:\Users\ashooshtari\AppData\Local\Temp\ipykernel_17108\3292534948.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_weather_hourly = df_weather.resample("1H").mean()
C:\Users\ashooshtari\AppData\Local\Temp\ipykernel_17108\3292534948.py:27: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_weather_hourly.fillna(method="ffill", inplace=True)
C:\Users\ashooshtari\AppData\Local\Temp\ipykernel_17108\3292534948.py:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_weather_hourly.fillna(method="bfill", inplace=True)


Timestamp                0
T                        0
P0                       0
P                        0
U                        0
Ff                       0
ff10                     0
VV                       0
Td                       0
cloud_condition_score    0
cloud_base_meters        0
wind_x                   0
wind_y                   0
wind_is_variable         0
wind_is_calm             0
weather_severity         0
dtype: int64

In [13]:
def add_lag_features(df):
    lag_cols = ['T', 'Td', 'U']
    
    for col in lag_cols:
        df[f'{col}_lag1h'] = df[col].shift(1)
        df[f'{col}_lag24h'] = df[col].shift(24)
    
    return df

df_weather_hourly = add_lag_features(df_weather_hourly)
df_weather_hourly

,Timestamp,T,P0,P,U,Ff,ff10,VV,Td,cloud_condition_score,...,wind_y,wind_is_variable,wind_is_calm,weather_severity,T_lag1h,T_lag24h,Td_lag1h,Td_lag24h,U_lag1h,U_lag24h
0,2023-01-01 00:00:00,5.0,742.60,746.30,93.0,5.0,5.0,9.55,4.0,4.0,...,-0.382683,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01 01:00:00,5.0,742.95,746.65,90.0,5.0,5.0,10.10,3.5,4.0,...,-0.815493,0.0,0.0,1.0,5.0,NaN,4.0,NaN,93.0,NaN
2,2023-01-01 02:00:00,4.5,743.30,747.00,93.0,6.0,6.0,10.10,3.5,3.5,...,-0.923880,0.0,0.0,1.0,5.0,NaN,3.5,NaN,90.0,NaN
3,2023-01-01 03:00:00,4.0,744.10,747.80,93.0,6.0,6.0,10.10,3.0,3.0,...,-0.961940,0.0,0.0,1.0,4.5,NaN,3.5,NaN,93.0,NaN
4,2023-01-01 04:00:00,3.5,744.10,747.80,90.0,5.5,5.5,10.10,2.0,2.0,...,-0.923880,0.0,0.0,1.0,4.0,NaN,3.0,NaN,93.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00,-6.0,758.50,762.40,80.0,2.5,2.5,10.10,-9.0,4.0,...,0.923880,0.0,0.0,1.0,-6.0,-0.5,-8.0,-1.5,86.0,93.0
8756,2023-12-31 20:00:00,-7.0,758.90,762.80,86.0,1.5,1.5,10.10,-9.0,4.0,...,0.815493,0.0,0.0,1.0,-6.0,-1.0,-9.0,-2.0,80.0,93.0
8757,2023-12-31 21:00:00,-7.5,758.90,762.80,89.5,1.0,1.0,10.10,-9.0,4.0,...,-0.017450,0.0,0.5,1.0,-7.0,-1.0,-9.0,-2.0,86.0,93.0
8758,2023-12-31 22:00:00,-8.0,759.25,763.15,93.0,0.5,0.5,8.05,-9.0,4.0,...,-0.026176,0.5,0.5,5.0,-7.5,-1.5,-9.0,-2.0,89.5,96.5


In [14]:
# Assuming Timestamp column is already datetime
df_weather_hourly["hour"] = df_weather_hourly["Timestamp"].dt.hour
df_weather_hourly["dayofweek"] = df_weather_hourly["Timestamp"].dt.dayofweek
df_weather_hourly["month"] = df_weather_hourly["Timestamp"].dt.month
df_weather_hourly["dayofyear"] = df_weather_hourly["Timestamp"].dt.dayofyear
df_weather_hourly["weekofyear"] = df_weather_hourly["Timestamp"].dt.isocalendar().week.astype(int)
df_weather_hourly["is_weekend"] = df_weather_hourly["dayofweek"].isin([5, 6]).astype(int)

Estoni_holidays = holidays.country_holidays('EE')
df_weather_hourly["is_holiday"] = df_weather_hourly["Timestamp"].dt.date.astype(str).isin(Estoni_holidays).astype(int)

#T * U → feeling temperature
df_weather_hourly['FT'] = df_weather_hourly['T'] * df_weather_hourly['U']
#Cyclical encoding

df_weather_hourly['hour_sin'] = np.sin(2 * np.pi * df_weather_hourly['hour'] / 24)
df_weather_hourly['hour_cos'] = np.cos(2 * np.pi * df_weather_hourly['hour'] / 24)

df_weather_hourly['dayofweek_sin'] = np.sin(2 * np.pi * df_weather_hourly['dayofweek'] / 7)
df_weather_hourly['dayofweek_cos'] = np.cos(2 * np.pi * df_weather_hourly['dayofweek'] / 7)

df_weather_hourly['month_sin'] = np.sin(2 * np.pi * df_weather_hourly['month'] / 12)
df_weather_hourly['month_cos'] = np.cos(2 * np.pi * df_weather_hourly['month'] / 12)

df_weather_hourly

,Timestamp,T,P0,P,U,Ff,ff10,VV,Td,cloud_condition_score,...,weekofyear,is_weekend,is_holiday,FT,hour_sin,hour_cos,dayofweek_sin,dayofweek_cos,month_sin,month_cos
0,2023-01-01 00:00:00,5.0,742.60,746.30,93.0,5.0,5.0,9.55,4.0,4.0,...,52,1,0,465.00,0.000000,1.000000,-0.781831,0.62349,5.000000e-01,0.866025
1,2023-01-01 01:00:00,5.0,742.95,746.65,90.0,5.0,5.0,10.10,3.5,4.0,...,52,1,0,450.00,0.258819,0.965926,-0.781831,0.62349,5.000000e-01,0.866025
2,2023-01-01 02:00:00,4.5,743.30,747.00,93.0,6.0,6.0,10.10,3.5,3.5,...,52,1,0,418.50,0.500000,0.866025,-0.781831,0.62349,5.000000e-01,0.866025
3,2023-01-01 03:00:00,4.0,744.10,747.80,93.0,6.0,6.0,10.10,3.0,3.0,...,52,1,0,372.00,0.707107,0.707107,-0.781831,0.62349,5.000000e-01,0.866025
4,2023-01-01 04:00:00,3.5,744.10,747.80,90.0,5.5,5.5,10.10,2.0,2.0,...,52,1,0,315.00,0.866025,0.500000,-0.781831,0.62349,5.000000e-01,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00,-6.0,758.50,762.40,80.0,2.5,2.5,10.10,-9.0,4.0,...,52,1,0,-480.00,-0.965926,0.258819,-0.781831,0.62349,-2.449294e-16,1.000000
8756,2023-12-31 20:00:00,-7.0,758.90,762.80,86.0,1.5,1.5,10.10,-9.0,4.0,...,52,1,0,-602.00,-0.866025,0.500000,-0.781831,0.62349,-2.449294e-16,1.000000
8757,2023-12-31 21:00:00,-7.5,758.90,762.80,89.5,1.0,1.0,10.10,-9.0,4.0,...,52,1,0,-671.25,-0.707107,0.707107,-0.781831,0.62349,-2.449294e-16,1.000000
8758,2023-12-31 22:00:00,-8.0,759.25,763.15,93.0,0.5,0.5,8.05,-9.0,4.0,...,52,1,0,-744.00,-0.500000,0.866025,-0.781831,0.62349,-2.449294e-16,1.000000


In [15]:
df_weather_hourly.to_csv("df_weather_hourly.csv", index=False)

In [16]:
# Check for duplicates in df_el
duplicates_df_el = df_el[df_el['Timestamp'].duplicated(keep=False)]
print("Duplicates in df_el:", duplicates_df_el)
# Remove duplicates, keeping the first occurrence
df_el = df_el.drop_duplicates(subset='Timestamp', keep='first')

# Convert timestamps to sets for comparison
timestamps1 = set(df_weather_hourly['Timestamp'])
timestamps2 = set(df_el['Timestamp'])
# Find missing timestamps
missing_in_df_weather = timestamps2 - timestamps1
missing_in_df_el = timestamps1 - timestamps2

print("Timestamps in df_el but not in df_weather:", missing_in_df_weather)
print("Timestamps in df_weather but not in df_el:", missing_in_df_el)

for missing_time in missing_in_df_el:

    new_row = pd.DataFrame({'Timestamp': [missing_time]})
    df_el = pd.concat([df_el, new_row]).sort_values('Timestamp').reset_index(drop=True)
    
    idx = df_el[df_el['Timestamp'] == missing_time].index[0]
    
    # Interpolate numeric columns (average of previous and next)
    for col in df_el.select_dtypes(include='number').columns:
        if col != 'Timestamp' and idx > 0 and idx < len(df_el) - 1:
            prev_val = df_el.at[idx-1, col]
            next_val = df_el.at[idx+1, col]
            if pd.notnull(prev_val) and pd.notnull(next_val):
                df_el.at[idx, col] = (prev_val + next_val) / 2
            elif pd.notnull(prev_val):
                df_el.at[idx, col] = prev_val
            elif pd.notnull(next_val):
                df_el.at[idx, col] = next_val   



Duplicates in df_el:                Timestamp    ICT  U06, U06A, U05B    OBS  U05, U04, U04B, GEO  \
7225 2023-10-29 03:00:00  132.4            100.1  6.320                 98.2   
7226 2023-10-29 03:00:00  133.7             99.7  6.254                 97.2   

      TEG   LIB   MEK   SOC  S01  D04  
7225    1  35.8  18.2  69.9  4.9  8.0  
7226    1  36.3  18.6  70.9  5.6  7.8  
Timestamps in df_el but not in df_weather: set()
Timestamps in df_weather but not in df_el: {Timestamp('2023-03-26 04:00:00'), Timestamp('2023-03-26 03:00:00')}


In [17]:
df_el.to_csv("df_el.csv", index=False)